In [54]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

In [55]:
df = pd.read_parquet("/home/jan/STAR_shortcut_OSN_paper/data/LSGG/landing_df.parquet")

In [56]:
df['runway'] = df['runway'].apply(lambda x: x[:-1] if len(x) == 3 else x)
df["star"] = df["navaid"] + df["runway"]

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime

# Step 1: Convert 'stop' column to datetime if it's not already in that format
df['stop'] = pd.to_datetime(df['stop'])

# Step 2: Round the time down to the nearest 30 minutes
df['stop_half_hour'] = df['stop'].dt.floor('30T').dt.time

# Step 3: Group by the rounded time and count the number of rows for each half-hour
half_hour_counts = df.groupby(['stop_half_hour', 'runway']).size().reset_index(name='count')

# Step 4: Create a stacked bar plot using plotly.express
fig = px.bar(
    half_hour_counts,
    x='stop_half_hour',  # x-axis represents the half-hour intervals
    y='count',  # y-axis represents the count
    color='runway',  # color represents the different categories in 'runway'
    title='Stacked Bar Chart for Half-Hour Intervals with Navaid Categories',
    labels={'stop_half_hour': 'Time of Day (Half-Hour Intervals)', 'count': 'Number of Rows'},
    barmode='stack'  # This makes the bars stacked
)

# Step 5: Customize x-axis to show only the hour in 'hh:mm' format, left-aligned
x_ticks = [time.strftime('%H:%M') for time in half_hour_counts['stop_half_hour'] if time.minute == 0]

# Shift the tick values slightly to the left by subtracting a small amount (e.g., 0.5)
tickvals = [i - 1.5 for i, time in enumerate(half_hour_counts['stop_half_hour']) if time.minute == 0]

# Set layout for the plot
fig.update_layout(
    title='Number of Rows for Each Half-Hour Interval Over 24 Hours',
    xaxis_title='Time of Day (Half-Hour Intervals)',
    yaxis_title='Number of Rows',
    xaxis = dict(
        tickmode = 'array',
        tickvals = tickvals,
        ticktext = x_ticks
    )
)

### Distributions

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create the subplots with a 2x2 grid
fig = make_subplots(
    rows=2, cols=2, 
    subplot_titles=("BELU3N Arrivals", "KINE2N Arrivals", "AKIT3R Arrivals", "LUSA2N Arrivals"),
    vertical_spacing=0.17,
    horizontal_spacing=0.1
)

# Define the full star distances and axis parametres for each star
stars = {
    'BELUS04': 92.8,
    'KINES04': 103.8,
    'AKITO22': 110.3,
    'LUSAR04': 87.9
}

yranges = {
    'BELUS04': [0, 3000],
    'KINES04': [0, 300],
    'AKITO22': [0, 1500],
    'LUSAR04': [0, 1200]
}

xranges = {
    'BELUS04': [20, 110],
    'KINES04': [60, 150],
    'AKITO22': [50, 120],
    'LUSAR04': [45, 130]
}

# Loop over each star to generate the subplots
for i, (star, star_distance) in enumerate(stars.items()):
    row = i // 2 + 1
    col = i % 2 + 1
    
    # Retrieve the data for the specific star and remove outliers
    temp_df = df.query(f"star == '{star}' and distance < 200")

    # Calculate the 99th percentile and median for the specific star
    percentile_99 = np.percentile(temp_df.distance, 95)
    median = np.median(temp_df.distance)
    
    # Add the histogram for the star
    fig.add_trace(
        go.Histogram(
            x=temp_df.distance,
            xbins=dict(
                start=0,
                end=int(max(df.distance)),
                size=2,
            ),
            marker=dict(color="#1f77b4"),
            name=f"Observed distances  ",
            showlegend=(i == 0),  # Show legend only for the first subplot
        ),
        row=row, col=col
    )

    # Add a line indicating the median of observed distances
    fig.add_trace(
        go.Scatter(
            x=[median, median],
            y=[-500, 3000],
            mode="lines",
            line=dict(color="darkblue", width=3, dash="dash"),
            name=f"Median of observed distances  ",
            showlegend=(i == 0),
        ),
        row=row, col=col
    )

    # Add a line for the 99th percentile of observed distances
    fig.add_trace(
        go.Scatter(
            x=[percentile_99, percentile_99],
            y=[-500, 3000],
            mode="lines",
            line=dict(color="#ff7f0e", width=3, dash="dash"),
            name=f"95th Percentile of observed distances  ",
            showlegend=(i == 0),
        ),
        row=row, col=col
    )

    # Add the line for "Full STAR distance" according to the procedure
    fig.add_trace(
        go.Scatter(
            x=[star_distance, star_distance],
            y=[-100, 3000],
            mode="lines",
            line=dict(color="#d62728", width=3, dash="dash"),
            name=f"Full STAR distance as per procedure  ",
            showlegend=(i == 0),
        ),
        row=row, col=col
    )

    # Update the layout for each subplot: title size, y-axis range, x-axis range
    fig.update_annotations(font_size=24)
    fig.update_yaxes(
        range=yranges[star],
        row=row, col=col,
    )
    fig.update_xaxes(
        range=xranges[star],
        row=row, col=col,
    )

# Update the x-axes and y-axes labels and ranges for each subplot
fig.update_xaxes(
    title_text="Distance flown [NM]",
    titlefont=dict(size=20),
    tickfont=dict(size=20),
)
fig.update_yaxes(
    title_text="Count",
    titlefont=dict(size=20),
    tickfont=dict(size=20),
)

# Update the layout
fig.update_layout(
    width=2000,
    height=1000,
    margin=dict(l=110, r=40, t=40, b=0),
    legend=dict(
        font=dict(size=24),
        yanchor="bottom",
        xanchor="center",
        x=0.5,
        y=-0.17,
        orientation="h",
        tracegroupgap=50
    )
)

# Show the figure
fig.show()


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

df = df.copy()
df["timestamp_lt"] = df["stop"].dt.tz_convert("Europe/Berlin")
df["hour_utc"] = df.stop.dt.hour
df["hour_lt"] = df.timestamp_lt.dt.hour

# Create the subplots with a 2x2 grid
fig = make_subplots(
    rows=2, cols=2, 
    subplot_titles=("BELU3N Arrivals", "KINE2N Arrivals", "AKIT3R Arrivals", "LUSA2N Arrivals"),
    vertical_spacing=0.17,
    horizontal_spacing=0.1
)

# Define the full star distances for each star
stars = {
    'BELUS04': 92.8,
    'KINES04': 103.8,
    'AKITO22': 110.3,
    'LUSAR04': 87.9
}

yranges = {
    'BELUS04': [20, 120],
    'KINES04': [40, 140],
    'AKITO22': [40, 140],
    'LUSAR04': [40, 120]
}

# Loop over each star to generate the subplots
for i, (star, star_distance) in enumerate(stars.items()):
    row = i // 2 + 1
    col = i % 2 + 1
    
    # Filter the data for the specific star
    temp_df_star = df.query(f"star == '{star}' and distance < 200")
    
    # Add required columns (for grouping by hour of the day)
    temp_df_star["timestamp_lt"] = temp_df_star["stop"].dt.tz_convert("Europe/Berlin")
    temp_df_star["hour_lt"] = temp_df_star.timestamp_lt.dt.hour

    # Create the box plot for each star's hourly distribution
    fig.add_trace(
        go.Box(
            x=temp_df_star["hour_lt"],
            y=temp_df_star["distance"],
            boxpoints="outliers",  # Show outliers
            name=f"Observed distances  ",
            marker=dict(color="#1f77b4"),
            showlegend=(i == 0),
        ),
        row=row, col=col,
    )

    fig.update_yaxes(
        range=yranges[star],
        row=row, col=col,
    )

    # Add line for full STAR distance
    fig.add_trace(go.Scatter(
        x=[-0.5, temp_df_star['hour_lt'].max() + 1],
        y=[star_distance, star_distance],
        mode="lines",
        line=dict(color="red", dash="dash", width=2),
        name=f"Full STAR distance as per procedure  ",
        showlegend=(i == 0),  # Show legend only in the first subplot
    ), row=row, col=col)


# Update the layout for each subplot: title size, y-axis range, x-axis range
fig.update_annotations(font_size=24)

# Set global axis parameters
fig.update_yaxes(
    title_text="Distance [NM]",
    titlefont=dict(size=20),
    tickfont=dict(size=20),
)
fig.update_xaxes(
    title_text="Hour of the day (local time)",
    titlefont=dict(size=20),
    tickfont=dict(size=20),
    tickmode="linear",
    dtick=1,  # Interval for each hour
)

# Update layout globally
fig.update_layout(
    width=2000,
    height=1000,
    margin=dict(l=110, r=40, t=40, b=0),
    legend=dict(
        font=dict(size=24),
        yanchor="bottom",
        xanchor="center",
        x=0.5,
        y=-0.17,
        orientation="h",
        tracegroupgap=50
    )
)

# Show the figure
fig.show()




In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime

# Step 1: Convert 'stop' column to datetime if it's not already in that format
df['stop'] = pd.to_datetime(df['stop'])

# Step 2: Round the time down to the nearest 30 minutes
df['stop_half_hour'] = df['stop'].dt.floor('30T').dt.time

# Step 3: Group by the rounded time and count the number of rows for each half-hour
half_hour_counts = df.groupby(['stop_half_hour', 'runway']).size().reset_index(name='count')

# Step 4: Create a stacked bar plot using plotly.express
fig = px.bar(
    half_hour_counts,
    x='stop_half_hour',  # x-axis represents the half-hour intervals
    y='count',  # y-axis represents the count
    color='runway',  # color represents the different categories in 'runway'
    title='Stacked Bar Chart for Half-Hour Intervals with Navaid Categories',
    labels={'stop_half_hour': 'Time of Day (Half-Hour Intervals)', 'count': 'Number of Rows'},
    barmode='stack'  # This makes the bars stacked
)

# Step 5: Customize x-axis to show only the hour in 'hh:mm' format, left-aligned
x_ticks = [time.strftime('%H:%M') for time in half_hour_counts['stop_half_hour'] if time.minute == 0]

# Shift the tick values slightly to the left by subtracting a small amount (e.g., 0.5)
tickvals = [i - 1.5 for i, time in enumerate(half_hour_counts['stop_half_hour']) if time.minute == 0]

# Set layout for the plot
fig.update_layout(
    title='Number of Rows for Each Half-Hour Interval Over 24 Hours',
    xaxis_title='Time of Day (Half-Hour Intervals)',
    yaxis_title='Number of Rows',
    xaxis = dict(
        tickmode = 'array',
        tickvals = tickvals,
        ticktext = x_ticks
    )
)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime

# Step 1: Convert 'stop' column to datetime if it's not already in that format
df['stop'] = pd.to_datetime(df['stop'])

# Step 2: Round the time down to the nearest 30 minutes
df['stop_half_hour'] = df['stop'].dt.floor('30T').dt.time

# Step 3: Group by the rounded time and count the number of rows for each half-hour
half_hour_counts = df.groupby(['stop_half_hour', 'navaid']).size().reset_index(name='count')

# Step 4: Create a stacked bar plot using plotly.express
fig = px.bar(
    half_hour_counts,
    x='stop_half_hour',  # x-axis represents the half-hour intervals
    y='count',  # y-axis represents the count
    color='navaid',  # color represents the different categories in 'navaid'
    title='Stacked Bar Chart for Half-Hour Intervals with Navaid Categories',
    labels={'stop_half_hour': 'Time of Day (Half-Hour Intervals)', 'count': 'Number of Rows'},
    barmode='stack'  # This makes the bars stacked
)

# Step 5: Customize x-axis to show only the hour in 'hh:mm' format, left-aligned
x_ticks = [time.strftime('%H:%M') for time in half_hour_counts['stop_half_hour'] if time.minute == 0]

# Shift the tick values slightly to the left by subtracting a small amount (e.g., 0.5)
tickvals = [i - 1.5 for i, time in enumerate(half_hour_counts['stop_half_hour']) if time.minute == 0]

# Set layout for the plot
fig.update_layout(
    title='Number of Rows for Each Half-Hour Interval Over 24 Hours',
    xaxis_title='Time of Day (Half-Hour Intervals)',
    yaxis_title='Number of Rows',
    xaxis = dict(
        tickmode = 'array',
        tickvals = tickvals,
        ticktext = x_ticks
    )
)

# Show the plot
fig.show()


In [26]:
import plotly.express as px

In [ ]:
stacked_data = df.groupby(['stop_half_hour', 'navaid']).size().reset_index(name='count')
stacked_data

In [ ]:
import plotly.express as px

df = px.data.gapminder().query("continent == 'Oceania'")
fig = px.bar(stacked_data, x='stop_half_hour', y='count',
             color='navaid', height=400)
fig.show()